# Introducing Pydantic: Pythonic Data Validation

### Goal of this notebook 
This notebook will introduce you to [Pydantic](https://docs.pydantic.dev/latest/), a data validation library that provides a way to define data schemas in a way that is both Pythonic and easy to use. 

**Note:** *We are using pydantic 2 in this tutorial! Many of the example will not work with pydantic 1.*


### Steps you will take in this notebook

1. Review creating a class in Python.
2. Learn about data classes in Python.
3. Learn how to write a class in Pydantic that enforces data types and constraints
4. Very briefly introduce `ipyautoui` to uncover an issue with our model of the controls.

## What problem are we trying to solve?

The dashboard we have built so far works, but would be hard to include as part of a larger project, and its individual components would be difficult to reuse. For example, the controls to select a range of years and apply smoothing to the data would be useful in making other graphs. As the code stand now, it would be hard to take those pieces as a unit and integrate them into a different dashboard.

Put differently, one might want a `DataSelector` widget that has as its value a range of years, a smoothing windows, and a polynomial order for the smoothing that can be reused.

### Our route to the solution is indirect

In this notebook we will define a class using Pydantic that does not, by itself, have any widgets attached to it. In the next notebook we will use `ipyautoui` to generate a widget-based user interface from our Pydantic class that will be much easier to reuse.

### Goals for our pydantic `DataSelector` class:

The class should have these attributes (also called *fields* in the pydantic documentation):

+ `year_range`, the range of time selected.
+ `window_size`, the size of the smoothing window.
+ `polynomial_order`, the order of the polynomial used in smoothing.

These attributes also have some important constraints:

+ The `window_size` should be an integer larger than one and, to match our earlier example, less than or equal to 100.
+ The `polynomial_order` should be an integer less than or equal to 10, and less than the `window_size`.


We import our dashboard below to get the magics from it.

In [1]:
import dashboard

## (optional) [Review of Python classes and a (First Draft) `DataSelector` Class](03az_OPTIONAL_intro_classes.ipynb)

## Constructing a (Second Draft) `DataSelector` Class with `dataclass`
Writing a class with multiple attributes gets repetitive in Python. Each attribute typically comes with an argument to the class and lines of boilerplate code to set the attributes of the class to the those arguments.

Data classes were introduced in Python 3.7 to make that sort of code more compact to write. They leverage *type annotations*, which were added to the language in version 3.0, and allow you to provide some information about the type of a variable.

The class below is an implemenation, using data classes, of part of the `DataSelectorPlainPython` we wrote above.

In [2]:
from dataclasses import dataclass

@dataclass 
class DataSelectorDataClass:
    """
    Partial implementation of a class to hold a data selector widget using dataclasses.
    """
    year_range: tuple = (1800, 2000)

This is already much more compact than the initial class definition above. Python automatically creates an `__init__` method for this class that sets the class up. It also comes with some extras:

In [ ]:
selector_dc = DataSelectorDataClass()

In [ ]:
print(selector_dc)

Compare this to what we get for our plain Python class:

In [ ]:
print(selector_plain)

As we will see in a few minutes, in addition to getting a nice string representation of the object for free, we also get the ability to test for equality of two instances.

### Exercise

1. Extend `DataSelectorDataClass` so that it also has a `window_size` with a default value of 2 and a `polynomial_order` with a default value of 1.

In [ ]:
# %answer key/03a/09.py

@dataclass
class DataSelectorDataClass:
    """
    A class to hold a data selector widget using dataclasses.
    """
    year_range: tuple = (1800, 2000)
    # Add your code below -- remember to include the default values


2. Recall that testing for equality did not work the way we wanted for the plain Python version of our data selector. Try comparing the two selectors below with each other and with `selector_dc` to see if equality testing works.

In [ ]:
sel_dc_2 = DataSelectorDataClass(year_range=(1991, 2018))
sel_dc_3 = DataSelectorDataClass(year_range=(1991, 2018))

In [ ]:
# %answer key/03a/11.py

# Put your comparison here

### Data class summary

Using `dataclass` to define our selector has several advantages:

+ It is less code.
+ It has a human-readable string representation.
+ You can check whether instances of the class are equal.

We could have done those last two things without `dataclass` by defining a couple of special methods methods in our class definition. However, it's really nice to just have it happen automatically behind the scenes, though!

*Note:* There is much more to data classes than we have covered. Read more about them in this [Real Python tutorial](https://realpython.com/python-data-classes/) or in the [Python documentation](https://docs.python.org/3/library/dataclasses.html).

## Progress check

The data class was relatively straightforward to write and looks promising for representing our controls:

```python
@dataclass 
class DataSelectorDataClass:
    """
    Partial implementation of a class to hold a data selector widget using dataclasses.
    """
    year_range: tuple = (1800, 2000)
    window_size: int = 2
    polynomial_order: int = 1
```

There are a couple of issues, though:

1. You can set any of the attributes to whatever value you want. This will raise no errors: .

In [ ]:
DataSelectorDataClass(year_range=5, window_size="three", polynomial_order=-3.14159)

2. None of the contraints we wanted are enforced.

Pydantic will help us solve these issues.

## Constructing a `DataSelector` Class with Pydantic

The `pydantic` library solves several of our problems and gets us a few more abilities for free:

+ It is designed to help enforce type requirements. It can do its best to convert values for you, or not if you prefer that.
+ Simple constraints like "this number must be greater than or equal to two" are easy to express.
+ More complicated constraints like "this number must be smaller than this other one" are possible to express.
+ It is straightforward to save objects to disk.

In [3]:
#| default_exp pydantic_model

### Making a class using pydantic

One way to use pydantic to make a class is to import a class called `BaseModel` from it and subclass that. Classes made using pydantic are often called *models*, a term we will use for pydantic-based classes. It ends up looking a lot like a data class:

In [4]:
# We will export the proper imports shortly, once we are closer to a final model
from pydantic import BaseModel

class DataSelectorModelDraft1(BaseModel):
    year_range: tuple = (1800, 2000)
    window_size: int = 2
    polynomial_order: int = 1

Like a data class, attributes are defined by adding a type annotation. Unlike data classes, pydantic enforces types. Try running the cell below, which will raise an exception:

In [5]:
%%exception

selector_pyd = DataSelectorModelDraft1(year_range=5, window_size="three", polynomial_order=-3.14159)

ValidationError: 3 validation errors for DataSelectorModelDraft1
year_range
  Input should be a valid tuple [type=tuple_type, input_value=5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/tuple_type
window_size
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='three', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing
polynomial_order
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=-3.14159, input_type=float]
    For further information visit https://errors.pydantic.dev/2.8/v/int_from_float

Pydantic includes fairly detailed error messages for validation errors. Many users could read this and update their input appropriately.

### Exercise

1. Make a valid instance of `DataSelectorModelDraft1`, i.e. an instance that does not raise an error when you create it. Feel free to try to come up with an instances that might surprise other people.

In [7]:
# %answer key/03a/13.py

selector_pyd = DataSelectorModelDraft1(year_range=(2020, 2022), window_size=10, polynomial_order=3)

### More ways to make an instance of a pydantic model

There a few ways of making an instance of a pydantic model:

1. Provide arguments when you call the class; this is what we did above.
2. From a dictionary of values, using the class method `model_validate`.
3. From json.

We will come back to the third way later in the notebook. An example of the second way is below.

In [8]:
my_choices = {
    "year_range": (1900, 1950),
    "window_size": 10,
    "polynomial_order": 2,
}

DataSelectorModelDraft1.model_validate(my_choices)

DataSelectorModelDraft1(year_range=(1900, 1950), window_size=10, polynomial_order=2)

### Imposing constraints after object creation

By default, pydantic imposes its constraints only when you create the object. Consider this example:

In [9]:
selector_pyd_simple = DataSelectorModelDraft1()
selector_pyd_simple.window_size = "two"
selector_pyd_simple

DataSelectorModelDraft1(year_range=(1800, 2000), window_size='two', polynomial_order=1)

Note that `window_size` has been set to a string, not an integer.

However, pydantic can be configured to check types when values are assigned by using the `validate_assignment` configuration. There are many more options available in [pydantic configuration](https://docs.pydantic.dev/latest/concepts/config/).

In [10]:
class DataSelectorModelDraft2(BaseModel, validate_assignment=True):
    year_range: tuple = (1800, 2000)
    window_size: int = 2
    polynomial_order: int = 1

Now an exception is raised when we try to assign a string to `window_size`.

In [11]:
%%exception

selector_pyd_simple2 = DataSelectorModelDraft2()
selector_pyd_simple2.window_size = "two"

ValidationError: 1 validation error for DataSelectorModelDraft2
window_size
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='two', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing

### More specific constraints on types

You might be surprised to see that the line below raises no error.

In [12]:
selector_pyd_simple2.year_range = ("eightteen eighty 5", 8+5j)

The reason is that pydantic simply checks to see that a tuple is being assigned to `year_range` -- the contents of the tuple can be anything at all. This would also raise no errors: `selector_pyd_simple2.year_range = (1, 2, 3)`

Python type annotations provide a way to provide information about what the tuple should consist of by putting the contents in square brackets, as shown in the cell below.

In [13]:
class DataSelectorModelDraft3(BaseModel, validate_assignment=True):
    year_range: tuple[int, int] = (1800, 2000)
    window_size: int = 2
    polynomial_order: int = 1

With this change, trying to assign `("eightteen eighty 5", 8+5j)` to `window_size` will fail.

In [14]:
%%exception

selector_pyd_simple3 = DataSelectorModelDraft3()
selector_pyd_simple3.year_range = ("eightteen eighty 5", 8+5j)

ValidationError: 2 validation errors for DataSelectorModelDraft3
year_range.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='eightteen eighty 5', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing
year_range.1
  Input should be a valid integer [type=int_type, input_value=(8+5j), input_type=complex]
    For further information visit https://errors.pydantic.dev/2.8/v/int_type

In [16]:
selector_pyd_simple4 = DataSelectorModelDraft3()
selector_pyd_simple4.year_range = (1, 2, 3)

ValidationError: 1 validation error for DataSelectorModelDraft3
year_range
  Tuple should have at most 2 items after validation, not 3 [type=too_long, input_value=(1, 2, 3), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.8/v/too_long

### Imposing constraints on field values

**Typing** refers to Python's type hinting system that allows you to specify the expected types of variables, function parameters, and return values. Also known as *type hinting*, this feature of Python helps reduce errors and enhance code readability. Later in the tutorial, we will learn more about a particular kind of type hinting called *type aliasing*, which will help us make more complex types easier to reuse and manage.

*Attributes in a pydantic model are typically called fields, terminology we will use for the remainder of the notebook.*

We have made some progress but we still have not imposed the constraints we want on window size and polynomial order. There are a couple new things we need to do that:

+ `Annotated` from Python's typing system lets you add additional information about the type of an item. Here we will use it to add information about the constraint on a field.
+ The `Field` class from pydantic is a class you use that contains that extra information. There are a number of possible arguments to `Field`. Here we use `ge`, short for "greater than or equal to," to impose the constraint that the `window_size` be larger greater than or equal to 2. The `Field` class from `pydantic` is somewhat similar to the [`field` class from Python's data classes](https://docs.python.org/3/library/dataclasses.html#dataclasses.field) which also serves the purpose of adding information about a typed field.

In the cell below we define a pydantic model that imposes the constraint that the `window_size` must be greater than or equal to 2. It does that by annotating the `window_size` type, `int`, with `Field(ge=2)`.

In [17]:
from typing import Annotated
from pydantic import Field

class DataSelectorModelDraft4(BaseModel, validate_assignment=True):
    year_range: tuple[int, int] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2)] = 2
    polynomial_order: int = 1

Let's test this by creating an instance and setting the `window_size` to an integer value that should not be allowed.

In [18]:
%%exception

selector_pyd_simple4 = DataSelectorModelDraft4()

selector_pyd_simple4.window_size = 0

ValidationError: 1 validation error for DataSelectorModelDraft4
window_size
  Input should be greater than or equal to 2 [type=greater_than_equal, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/greater_than_equal

Recall that the `window_size` also had an upper limit of 100 in the earlier dashboard we are trying to reproduce. This version of the class adds that upper limit also.

In [19]:
class DataSelectorModelDraft5(BaseModel, validate_assignment=True):
    year_range: tuple[int, int] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    polynomial_order: int = 1

#### Exercise

Add a constraint on the `polynomial_order` that requires is to be greater than or equal to 1 and less than or equal to 10.

In [20]:
# %answer key/03a/15.py

class DataSelectorModelDraft5(BaseModel, validate_assignment=True):
    year_range: tuple[int, int] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    # Modify the line below
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1


With these changes we have some of the constraints we want. 

### Imposing constraints on multiple fields

There is one more thing we need to do: the polynomial order should be at least 1 less than the window size in addition to being 10 or smaller.

To do that we will add a *model validator* to our pydantic class. The model validator has access to all of the proposed model values and can check them in whatever way it wants. If the values are acceptable then the method returns `self`. If the values are not acceptable then the validator should raise a `ValueError`.

A draft class with the model validator is below.

Now that we have a little experience with Pydantic we will start exporting the cells we need to make this part of our dashboard package.

In [21]:
#| export

from typing import Annotated
from pydantic import model_validator, BaseModel, Field

We are not quite ready to export our pydantic model -- we will do that later.

In [22]:
class DataSelectorModelDraft6(BaseModel, validate_assignment=True):
    year_range: tuple[int, int] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

#### Exercise

1. Check whether the new validation works by creating a valid `DataSelectorModelDraft6` and then trying to set the `polynomial_order` or `window_size` to inconsistent values. 

In [24]:
# %answer key/03a/17.py

# Break things here!
instance_model_6 = DataSelectorModelDraft6()
instance_model_6.window_size = 5
instance_model_6.polynomial_order = 7

ValidationError: 1 validation error for DataSelectorModelDraft6
  Value error, Polynomial order must be smaller than window size [type=value_error, input_value=DataSelectorModelDraft6(y...e=5, polynomial_order=7), input_type=DataSelectorModelDraft6]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

## Additional benefits of using Pydantic

### Easy to save model value to a file

One additional benefit of using pydantic to model our control is that pydantic classes come with easy conversion to json, which is in turn easy to save to disk.

We make a model in the cell below.

In [25]:
model = DataSelectorModelDraft6()

We can convert this model to a couple of different forms:

+ The `model_dump` method converts the pydantic model to a dictionary of values.
+ The `model_dump_json` method converts the pydantic model to json with the model's values.

In [26]:
print(model.model_dump())

{'year_range': (1800, 2000), 'window_size': 2, 'polynomial_order': 1}


In [27]:
# the indent argument causes the json to have line breaks, with the indentation
# of each new level given by indent
print(model.model_dump_json(indent=2))

{
  "year_range": [
    1800,
    2000
  ],
  "window_size": 2,
  "polynomial_order": 1
}


You might not be surprised to learn that you can also create a model instance from json. In the cell below, we take the json from `model` and use it to create a new instance.

To do that you use the class method `model_validate_json` to make the model.

In [28]:
model_json = model.model_dump_json()

# Use the class method model_validate_json to make a new model
new_model = DataSelectorModelDraft6.model_validate_json(model_json)

new_model

DataSelectorModelDraft6(year_range=(1800, 2000), window_size=2, polynomial_order=1)

In [29]:
with open("my_selections.json", "w") as f:
    f.write(model_json)

Though we will not have occasion to use it much in this tutorial, the next cell shows how to load the json from disk make a model from it.

In [30]:
with open("my_selections.json") as f:
    disk_json = f.read()

DataSelectorModelDraft6.model_validate_json(disk_json)

DataSelectorModelDraft6(year_range=(1800, 2000), window_size=2, polynomial_order=1)

### Text representation of the model

The `model_json_schema` method produces a json schema for the model, which is a description of the model and its restrictions.

While we will not have much occasion to use the json schema directly, it is really important to understand how the next package we will take a look at, `ipyautoui`, generates a user interface from a pydantic model.

The json schema for `DataSelectorModelDraft6` is below:

In [34]:
model.model_json_schema()

{'properties': {'year_range': {'default': [1800, 2000],
   'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'integer'}, {'type': 'integer'}],
   'title': 'Year Range',
   'type': 'array'},
  'window_size': {'default': 2,
   'maximum': 100,
   'minimum': 2,
   'title': 'Window Size',
   'type': 'integer'},
  'polynomial_order': {'default': 1,
   'maximum': 10,
   'minimum': 1,
   'title': 'Polynomial Order',
   'type': 'integer'}},
 'title': 'DataSelectorModelDraft6',
 'type': 'object'}

We can add extra keys to the json schema for a field by adding them a dictionary and using that dictionary as an argument in `Field`. The example below adds a key to the schema for `polynomial_order` noting that its value should be constrained by `window_size`, information not currently in the json schema.

In [35]:
# Make a dictionary....
schema_extra = dict(constraint_note="Polynomial order should be less than window size")


class DataSelectorModelDraftWithExtra(BaseModel):
    year_range: tuple[int, int] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    # ...provide the dictionary to json_schema_extra
    polynomial_order: Annotated[int, Field(ge=1, le=10, json_schema_extra=schema_extra)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

In [36]:
DataSelectorModelDraftWithExtra.model_json_schema()

{'properties': {'year_range': {'default': [1800, 2000],
   'maxItems': 2,
   'minItems': 2,
   'prefixItems': [{'type': 'integer'}, {'type': 'integer'}],
   'title': 'Year Range',
   'type': 'array'},
  'window_size': {'default': 2,
   'maximum': 100,
   'minimum': 2,
   'title': 'Window Size',
   'type': 'integer'},
  'polynomial_order': {'constraint_note': 'Polynomial order should be less than window size',
   'default': 1,
   'maximum': 10,
   'minimum': 1,
   'title': 'Polynomial Order',
   'type': 'integer'}},
 'title': 'DataSelectorModelDraftWithExtra',
 'type': 'object'}

## A problem with the data selector so far

There is a problem with our data selector class that is not obvious. To expose, we are going to use `ipyautoui`, a package that can generate a widget from a Pydantic model, to make a user interface from our model.

We will talk about ipyautoui in more detail in the next part of the tutorial. 

In [37]:
from ipyautoui import AutoUi

In [39]:
AutoUi(DataSelectorModelDraft6)

AutoUi(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style='success', disabled=True, icon…

There are two surprising things here. The first is that there was a validation error for `year_range`. The second is that the control for `year_range` does not look anything like a way to enter a range of years.

Resolving the second issue will also, it turns out, address the first.

The problem here is that our description of `year_range` is incomplete. Right now we annotate `year_range` as `tuple[int, int]`. That isn't quite right, since there is a constraint on both the first and second integer. Each must be in the range of years covered by the data.

### Get the year range from the data

One other issue is that we have manually put in the start and end year. Let's fix that up now by importing the data to get the range of years.

In [40]:
#| export

from pathlib import Path
import pandas as pd

In [41]:
#| export

DATA_DIR = 'data'
DATA_FILE = 'land-ocean-temp-index.csv'

original_data = pd.read_csv(Path(DATA_DIR) / DATA_FILE, escapechar='#')
min_year = original_data['Year'].min()
max_year = original_data['Year'].max()

## Fixing the slider problem: adding a constraint to `year_range` contents

We want to express to pydantic that each `int` has a restricted range. We'll do that by defining our own type below and then using that custom type in our Pydantic model.

We use a feature of type hinting introduced in Python 3.12 called type aliasing to do this. 

Since this tutorial was written to work in Python 3.11, we need to use the `typing-extensions` package, which is a package that back ports new typing features in Python to versions before the feature was added.

The cell below imports`TypeAliasType` from `typing-extensions`to let pydantic and type checkers know that we are defining a new type.

In [42]:
#| export

from typing_extensions import TypeAliasType

Next we define a `ConstrainedInt` type that includes limits on its range. Instead of using fixed values of 1880 and 2023 we read in the data file and use the minimum and maximum from the data.

In [45]:
#| export

ConstrainedInt = TypeAliasType(
    "ConstrainedInt", 
    Annotated[
        int, 
        Field(ge=min_year, le=max_year)
    ]
)

**Note:** Our kernel is running Python 3.11. The above feature looks very different in Python 3.12 than it does in this and earlier versions of Python. The equivalent to the definition of `ConstrainedInt` in Python 3.12 looks like this:

```python
# Only works in Python 3.12 and up!
type ConstrainedInt = Annotated[int, Field(ge=1880, le=2023)]
```

Finally, we write our pydantic model using this new type to `year_range`.

In [46]:
#| export

class DataSelectorModel(BaseModel, validate_assignment=True):
    year_range: Annotated[
        # The key change is in the line below
        tuple[ConstrainedInt, ConstrainedInt],
        # With this change to the type we no longer need to tell ipyautoui
        # what kind of widget to use. Field contains just a brief description
        Field(description="Range of years to plot")
    ] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

### Exercise: try out the auto-generated user interface

Make an interface using `AutoUi` from the model above.

In [47]:
# %answer key/03a/19.py
AutoUi(DataSelectorModel)

AutoUi(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style='success', disabled=True, icon…

Looks great! We will save our progress here and look into ipyautoui more in the next part of the tutorial.

## Export module with `nbdev`

We save the code we want to reuse in the cell below.

In [48]:
from nbdev.export import nb_export

nb_export('03a_pydantic.ipynb', 'dashboard_pydantic')

## FYI: `pydantic.TypeAdapter` can make any class a Pydantic class

Pydantic 2 introduced `TypeAdapter` ([docs are here](https://docs.pydantic.dev/latest/concepts/type_adapter/) and it is also discussed in the [Types documentation ](https://docs.pydantic.dev/latest/concepts/types/#constrained-types)) which essentially takes any class and adds some of the methods you get if you subclass from `BaseModel`. This is very useful if the class you would like to work with is one from a package that you cannot modify to subclass from `BaseModel`.